In [5]:
import numpy as np
import pandas as pd
import faiss
import torch
from transformers import AutoModel, AutoTokenizer
from bert import BERT

model_path = '/Users/user/KB_AI/KB_similarity/KB_ALBERT'

model = BERT(AutoModel.from_pretrained(model_path))
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of the model checkpoint at /Users/user/KB_AI/KB_similarity/KB_ALBERT were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
sts_dataset = pd.read_csv('../Dataset/kb_dataset_sts_all.csv').drop('Unnamed: 0',axis=1)

In [7]:
sts_dataset.iloc[:3]

,sent1,sent2,score
0,'[Company]에서는 첫 자산 연동 고객을 대상으로 프로모션 코드 입력 시 즉시...,"'**은행에서는 고객님께 자산관리, 지출관리, 금융플러스 서비스 등을 제공하는 마이...",3.5
1,"'[[Company]]에서는 발행어음 가입 이벤트를 진행 중으로, 1천만원 이상 순...",해당 카드를 이용해 총 100만원 이상 결제 시 2만원 캐시백을 받을 수 있는 이벤...,2.5
2,'**은행에서는 단 하루 동안 스마트폰뱅킹 서비스를 이용하는 고객에게 아메리카노 모...,'**은행'에서 직장인 고객을 대상으로 급여이체를 통해 다양한 혜택을 제공하는 이벤...,2.0


In [9]:
from numpy import dot
from numpy.linalg import norm

def convert_to_tensor(corpus, tokenizer):
    inputs = tokenizer(corpus,
                       truncation=True,
                       return_tensors="pt",
                       max_length=50,
                       padding="max_length")
    
    embedding = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']
        
    inputs = {'source': torch.LongTensor(embedding),
              'token_type_ids': torch.LongTensor(token_type_ids),
              'attention_mask': attention_mask}
    
    return inputs

def cos_sim(df):
  A = df.sent1
  B = df.sent2
  query_embedding_A = model.encode(convert_to_tensor(A, tokenizer))
  query_embedding_A = query_embedding_A.detach().numpy()

  query_embedding_B = model.encode(convert_to_tensor(B, tokenizer))
  query_embedding_B = query_embedding_B.detach().numpy()

  cos_sim = dot(query_embedding_A, query_embedding_B.T)/(norm(query_embedding_A)*norm(query_embedding_B))
  cos_sim = round(cos_sim[0][0],2)
  return cos_sim
sts_dataset['cosine_similarity'] = sts_dataset[:3740].apply(cos_sim,axis=1)
sts_dataset['normalized_score'] = sts_dataset['score'].apply(lambda x: round(x/5,2))
    

In [ ]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr

print('score와 cosine similarity 사이 pearson상관관계:',round(pearsonr(sts_dataset.iloc['cosine_similarity'],sts_dataset.iloc['normalized_score'])[0],4))
print('score와 cosine similarity 사이 spear상관관계:',round(spearmanr(sts_dataset.iloc['cosine_similarity'],sts_dataset.iloc['normalized_score'])[0],4))